## Capstone Project - IBM Data Science Certification on Coursera

### Week 3

## Introduction

Exploring and clustering the neighborhoods of Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scrape wikipedia list into a clean dataframe</a>

2. <a href="#item2">Determine latitude/longitude coordinates</a>

3. <a href="#item3">Cluster the Toronto neighborhoods</a>
  
</font>
</div>

Load standard and specialized libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# for http requests and JSON
import requests
import json
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

# for translating addresses and postal codes to latitude/longitude
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
#!pip install geocoder
import geocoder

# for map rendering
#!pip install folium==0.5.0
import folium

# for final clustering step
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Scrape wikipedia website and Create a clean dataframe

Test scraping with pandas

In [2]:
tmp = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
type(tmp)

list

Is returning as a list, so just need first index

In [3]:
tor_df = tmp[0]
tor_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Clean and filter the dataframe

In [4]:
# rename PostalCode column

tor_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
tor_df.head(1)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned


In [5]:
# ignore cells with a Borough of 'Not assigned'

idx = tor_df[tor_df['Borough'] == 'Not assigned'].index
cnt = idx.size

if cnt > 0:
    print('Dropping {} unassigned Boroughs'.format(cnt))
    tor_df.drop(idx, inplace=True)
    tor_df.reset_index(drop=True, inplace=True)
else:
    print("No unassigned Boroughs to drop")

Dropping 77 unassigned Boroughs


In [6]:
# merge Neighborhoods if duplicate PostalCodes exist

dup = tor_df.duplicated(subset='PostalCode', keep='first')
idx = dup[dup].index   # 'True' rows from duplicate check
cnt = idx.size

if cnt > 0:
    print('Merging {} duplicate PostalCode entries'.format(cnt))
    for i in idx:
        # find first kept entry and append to its Neighborhood list
        fdx = tor_df.loc[tor_df['PostalCode'] == tor_df.iloc[i]['PostalCode']].index[0]
        tor_df.iloc[fdx]['Neighborhood'] = tor_df.iloc[fdx]['Neighborhood'] + ', ' + tor_df.iloc[i]['Neighborhood']
        
    # drop the duplicate rows
    tor_df.drop(idx, inplace=True)
    tor_df.reset_index(drop=True, inplace=True)
else:
    print("No duplicate PostalCodes exist in current version of wikipedia")

No duplicate PostalCodes exist in current version of wikipedia


In [7]:
# check for 'Not assigned' Neighborhoods

idx = tor_df[tor_df['Neighborhood'] == 'Not assigned'].index
cnt = idx.size

if cnt > 0:
    print('Updating {} unassigned Neighborhoods'.format(cnt))
    for i in idx:
        # replace 'Not assigned' with name of Borough
        tor_df.iloc[i]['Neighborhood'] = tor_df.iloc[i]['Borough']
else:
    print("No unassigned Neighborhoods to update")

No unassigned Neighborhoods to update


Compare a subset of the results to the example in the assignment write-up

In [8]:
hw_pcs = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
hw_idx = tor_df['PostalCode'].isin(hw_pcs)
sub_df = tor_df[hw_idx]
sub_df

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


And report the final shape/number of rows in the full dataframe

In [9]:
print('Final cleaned dataframe contains {} rows'.format(tor_df.shape[0]))

Final cleaned dataframe contains 103 rows


<a id='item2'></a>

## 2. Determine neighborhood latitude/longitude coordinates

Load csv PostalCode listing as a backup to Geocoder

In [10]:
ref_df = pd.read_csv('https://cocl.us/Geospatial_data')
print('Backup reference loaded for use in case Geocoder fails')

Backup reference loaded for use in case Geocoder fails


In [11]:
ref_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Use Geocoder (or csv backup) to find the corresponding latitude/longitude for each PostalCode

In [12]:
cnt = 0   # counter to track success of Geocoder usage

lat_list = []
lng_list = []

for i in tor_df.index:
    pc = tor_df.iloc[i]['PostalCode']
    
    # try Geocoder twice in each loop iteration before failover
    
    coords = None
    adr = '{}, Toronto, Ontario'.format(pc)
    g = geocoder.google(adr)
    if coords is None:
        g = geocoder.arcgis(adr)   # additional attempt with different provider
    
    if coords is None:   # still
        # fall back to csv reference if necessary
        ref_i = ref_df.loc[ref_df['Postal Code'] == pc].index[0]
        lat = ref_df.iloc[ref_i]['Latitude']
        lng = ref_df.iloc[ref_i]['Longitude']
    else:
        # Geocoder successful
        coords = g.latlng
        lat = coords[0]
        lng = coords[1]
        cnt = cnt + 1

    # update lists with new column values
    lat_list.append(lat)
    lng_list.append(lng)
    
print('Geocoder calls succeeded {} times'.format(cnt))

# add coordinates as additional columns to dataframe from Part 1

tor_df.insert(3, 'Latitude', lat_list)
tor_df.insert(4, 'Longitude', lng_list)

Geocoder calls succeeded 0 times


Build the same subset as in Part 1 to check values against the assignment write-up

In [13]:
sub_df = tor_df[hw_idx]
sub_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849


<a id='item3'></a>

## 3. Explore and Cluster the Toronto neighborhoods

Focusing on the Boroughs that contain the word 'Toronto'

In [14]:
idx = tor_df[tor_df['Borough'].str.contains('Toronto')].index
neighborhoods = tor_df.iloc[idx]
neighborhoods.reset_index(drop=True, inplace=True)

print('Turning focus to the {} Boroughs that contain the word "Toronto"'.format(idx.size))
neighborhoods.head()

Turning focus to the 39 Boroughs that contain the word "Toronto"


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [15]:
print('The dataframe for mapping and clustering now contains {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
            neighborhoods.shape[0]
     )
)

The dataframe for mapping and clustering now contains 4 boroughs and 39 neighborhoods.


In order to define an instance of the geocoder, we need to define a user_agent.

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create initial map of Toronto with neighborhoods superimposed on top.

In [17]:
# create a map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

 Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'Y3MVHWR3QCOO3UAIXS2BHXRPMLPQFN4ILS50KNTHQ0EIZHFR'
CLIENT_SECRET = 'KRGBTZB5VSVXNNLYX2A2JDO4VGELMFOKN2YUMH5YGWI11ZIF' # reset after submitting assignment
VERSION = '20180605' # Foursquare API version

Explore the first neighborhood in our dataframe.

In [19]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Get the top 100 venues that are in the first neighborhoods within a radius of 500 meters.

In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request

In [21]:
results = requests.get(url).json()

Borrow the **get_category_type** function from the Foursquare lab.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a *pandas* dataframe.

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Dominion Pub and Kitchen,Pub,43.656919,-79.358967


Create a function to repeat the same process to all of the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Code to run the above function on each neighborhood and create a new dataframe called *tor_venues*.

In [25]:
tor_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                             latitudes=neighborhoods['Latitude'],
                             longitudes=neighborhoods['Longitude']
                            )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Check the size of the resulting dataframe

In [26]:
print(tor_venues.shape)
tor_venues.head()

(1621, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


Check how many venues were returned for each neighborhood

In [27]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [29]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

Examine the new dataframe size.

In [30]:
tor_onehot.shape

(1621, 236)

Group rows by neighborhood and take the mean of the frequency of occurrence of each category

In [31]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.0000,0.017544,0.0000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.0000,0.000000,0.0000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.0000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.017544,0.052632,0.087719,0.000000,0.000000,0.000000,0.00,0.000000,0.017544,0.000000,0.017544,0.000000,0.00,0.000000,0.00000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000

Confirm the new size

In [32]:
tor_grouped.shape

(39, 236)

Print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3      Beer Bar  0.04
4        Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.14
1         Breakfast Spot  0.09
2            Coffee Shop  0.09
3     Italian Restaurant  0.05
4  Performing Arts Venue  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.06
1               Brewery  0.06
2  Gym / Fitness Center  0.06
3         Garden Center  0.06
4                Garden  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay Street----
                 venue  freq
0

Function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Park,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Furniture / Home Store,Burrito Place,Restaurant,Stadium,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Salad Place,Thai Restaurant,Department Store,Bar,Burger Joint


Run *k*-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Farmers Market,Chocolate Shop,Gym / Fitness Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Tea Room,Electronics Store,Pizza Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Gym,Hotel,Italian Restaurant,Japanese Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine each cluster and look for the discriminating venue categories that distinguish it from the others

#### Cluster 1

In [39]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Farmers Market,Chocolate Shop,Gym / Fitness Center
1,Downtown Toronto,0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Japanese Restaurant,Café,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Tea Room,Electronics Store,Pizza Place
3,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Restaurant,American Restaurant,Cocktail Bar,Gym,Hotel,Italian Restaurant,Japanese Restaurant
4,East Toronto,0,Trail,Health Food Store,Pub,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Café,Bakery,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Park,Beach
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Salad Place,Thai Restaurant,Department Store,Bar,Burger Joint
7,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Nightclub,Restaurant,Italian Restaurant,Diner,Candy Store,Coffee Shop
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Hotel,Deli / Bodega,Thai Restaurant,Pizza Place,Cosmetics Shop,Salad Place
9,West Toronto,0,Bakery,Pharmacy,Grocery Store,Pizza Place,Café,Supermarket,Bar,Bank,Middle Eastern Restaurant,Brewery


#### Cluster 2

In [40]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Lawyer,Yoga Studio,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


#### Cluster 3

In [41]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
33,Downtown Toronto,2,Park,Playground,Trail,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


#### Cluster 4

In [42]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Swim School,Bus Line,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Cluster 5

In [43]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,Music Venue,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


# Observations

Most downtown Toronto neighborhoods clustered into the same cluster, dominated by an apparent high interest in Coffee Shops and Cafes!  The other small clusters show a concentration of Lawyer offices and Parks.  Could potentially increase the number of clusters for further analysis.

Part 3 of this notebook uses material originally created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/).